In [1]:
try:
    from IPython import get_ipython
    get_ipython().magic('clear')
    get_ipython().magic('reset -f')
except:
    pass

In [2]:
#Standard libraries for data analysis:
    
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#from pandas import Series,DataFrame
from scipy.stats import norm, skew
from scipy import stats
import statsmodels.api as sm

#Miscellaneous Utilitiy Libraries:
    
import random
import os
import re
import sys
import timeit
import string
import time
from datetime import datetime
from time import time
from dateutil.parser import parse
import joblib

In [3]:
df = pd.read_csv('subscriber_subset.csv')

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.describe()

In [ ]:
df.dtypes

In [4]:
df.columns.to_series().groupby(df.dtypes).groups

{int64: ['org_id', 'endpoint_id', 'status', 'current'], object: ['date_time']}

In [5]:
df.info

<bound method DataFrame.info of        org_id  endpoint_id  status  current  date_time
0      160090     74002331       3        1  12/2/2017
1      160090     85592712    1003        1  5/19/2018
2      160090     86164407    1003        1  5/28/2018
3      160090     75821508    1003        1  6/28/2018
4      160090     87939366    1003        1  6/29/2018
...       ...          ...     ...      ...        ...
81188  160090     88520174    1003        1  7/16/2018
81189  160090     88520174    1003        1  7/10/2018
81190  160090     89676934    1003        1  7/31/2018
81191  160090     88520174       4        1  7/17/2018
81192  160090     77725358    1003        1  7/30/2018

[81193 rows x 5 columns]>

In [ ]:
df.isna().any()

In [6]:
df["current"].value_counts()

1     77383
0      2553
2      1084
16       70
3        55
4        37
5         5
6         2
14        2
7         1
17        1
Name: current, dtype: int64

In [ ]:
df2 = df[['org_id','endpoint_id','status','current']]

#histogram

fig = plt.figure(figsize =(15,12))
plt.suptitle('Histograms of Numeric Columns\n' ,horizontalalignment="center",fontstyle = "normal", fontsize = 24, fontfamily = "sans-serif")
for i in range(df2.shape[1]):
    plt.subplot(6,3,i+1)
    f=plt.gca()
    f.set_title(df2.columns.values[i])
    
vals = np.size(df2.iloc[:, i].unique())
if vals >= 100:
    vals = 100
    
plt.hist(df2.iloc[:, i], bins=vals, color = '#ec838a')
plt.tight_layout(rect=[0, 0.03, 1, 0.95])

In [ ]:
df2.shape

In [ ]:
df.shape[0], df.endpoint_id.nunique()

In [ ]:
df.groupby(['endpoint_id']).count()

In [ ]:
df.query('endpoint_id == 60637549')

In [ ]:
df.query('endpoint_id == 60637549 & status == 4 | status = 3')

In [7]:
df1 = df[
    (df['endpoint_id'] == 60637549) & 
    (
      (
        (df['status'] ==4)
        &
        (df['current'] ==0)
      )
     |
      ( 
        (df['status'] == 3) 
       &
       (df['current'] ==1)
     )
    )
   ].sort_values(by=['date_time'])

In [8]:
#df_60637549
df1

,org_id,endpoint_id,status,current,date_time
2145,160090,60637549,4,0,1/24/2018
2723,160090,60637549,3,1,2/7/2018
36506,160090,60637549,4,0,6/21/2018
36606,160090,60637549,3,1,6/22/2018


In [9]:
df1.date_time.max()

'6/22/2018'

In [16]:
df1.groupby(["endpoint_id"]).agg({'date_time' : np.max})

,date_time
endpoint_id,
60637549,2018-06-22


In [ ]:
df1 = df1.groupby(["status","endpoint_id"])
df1 = df1.size()
df.reset_index(name='Count')

In [10]:
df1['date_time'] = pd.to_datetime(df1['date_time'])
df1 = (df1[df1['endpoint_id'].ne(0)]
         .sort_values(['endpoint_id','date_time'])
         .drop_duplicates('endpoint_id', keep='last'))
print(df1)

       org_id  endpoint_id  status  current  date_time
36606  160090     60637549       3        1 2018-06-22


In [11]:
df2 = df[
    #(df['endpoint_id'] == 60637549) & 
    (
      (
        (df['status'] ==4)
        &
        (df['current'] ==0)
      )
     |
      ( 
        (df['status'] == 3) 
       &
       (df['current'] ==1)
     )
    )
   ].sort_values(by=['date_time'])

In [13]:
df2['date_time'] = pd.to_datetime(df2['date_time'])
df2 = (df2[df2['endpoint_id'].ne(0)]
         .sort_values(['endpoint_id','date_time'])
         .drop_duplicates('endpoint_id', keep='last'))
print(df2)

       org_id  endpoint_id  status  current  date_time
1005   160090      2631826       3        1 2017-12-02
3572   160090      2631844       3        1 2017-12-02
3573   160090      2631853       3        1 2017-12-02
1006   160090      2631858       3        1 2017-12-02
18     160090      2631877       3        1 2017-12-02
...       ...          ...     ...      ...        ...
80790  160090     89242745       4        0 2018-07-27
79714  160090     89432947       3        1 2018-07-26
70426  160090     89442940       3        1 2018-07-26
80789  160090     89444982       3        1 2018-07-27
74454  160090     89708058       3        1 2018-08-01

[10878 rows x 5 columns]


In [14]:
df2.to_csv('output.csv')

In [18]:
df2.insert(5,"Churn_status","",allow_duplicates=True)

In [19]:
df2

,org_id,endpoint_id,status,current,date_time,Churn_status
1005,160090,2631826,3,1,2017-12-02,
3572,160090,2631844,3,1,2017-12-02,
3573,160090,2631853,3,1,2017-12-02,
1006,160090,2631858,3,1,2017-12-02,
18,160090,2631877,3,1,2017-12-02,
...,...,...,...,...,...,...
80790,160090,89242745,4,0,2018-07-27,
79714,160090,89432947,3,1,2018-07-26,
70426,160090,89442940,3,1,2018-07-26,
80789,160090,89444982,3,1,2018-07-27,


In [29]:
df2.loc[df2['status'] == 3, 'Churn_status'] = 'Churned'
df2.loc[df2['status'] == 4, 'Churn_status'] = 'Not Churned'

In [30]:
df2

,org_id,endpoint_id,status,current,date_time,Churn_status
1005,160090,2631826,3,1,2017-12-02,Churned
3572,160090,2631844,3,1,2017-12-02,Churned
3573,160090,2631853,3,1,2017-12-02,Churned
1006,160090,2631858,3,1,2017-12-02,Churned
18,160090,2631877,3,1,2017-12-02,Churned
...,...,...,...,...,...,...
80790,160090,89242745,4,0,2018-07-27,Not Churned
79714,160090,89432947,3,1,2018-07-26,Churned
70426,160090,89442940,3,1,2018-07-26,Churned
80789,160090,89444982,3,1,2018-07-27,Churned


In [31]:
df2.to_csv('output.csv')